In [1]:
#common code:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import re
#week 3
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#week 4
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
#week 5
import requests
from bs4 import BeautifulSoup
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#week 6
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor
from sklearn.svm import LinearSVC, SVC, LinearSVR
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier


In [2]:
train = pd.read_csv('./data/clean_train.csv')

In [3]:
train.head() #preview data

,animalid,name,datetime,outcometype,outcomesubtype,animaltype,sexuponoutcome,ageuponoutcome,breed,color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,unknown,Dog,Neutered Male,365,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,365,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,730,Pit Bull Mix,Blue/White
3,A683430,Unnamed,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,21,Domestic Shorthair Mix,Blue Cream
4,A667013,Unnamed,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,730,Lhasa Apso/Miniature Poodle,Tan


In [4]:
train['outcometype'].value_counts(normalize=True) #baseline

Adoption           0.403530
Transfer           0.352231
Return_to_owner    0.179301
Euthanasia         0.058156
Died               0.006782
Name: outcometype, dtype: float64

In [5]:
features = ['animaltype', 
            'sexuponoutcome', 'ageuponoutcome', 'breed', 'color']
X = train[features]
X = pd.get_dummies(X, columns=['animaltype', 'sexuponoutcome', 'breed', 'color'],
                  drop_first=True)
y = train['outcometype']
y = pd.get_dummies(y)

In [6]:
X.shape

(26687, 1750)

My dummification made quite a few more columns (probably names mostly) in X. I don't need to scale because my only integer value is all on the same scale (days).

In [7]:
y.shape

(26687, 5)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=17,stratify=y)

In [9]:
for num in [X_train, X_test, y_train, y_test]: #check shapes
    print(num.shape)

(20015, 1750)
(6672, 1750)
(20015, 5)
(6672, 5)


My first model will be a Random Forest classifier.

In [10]:
rf = RandomForestClassifier()
params = {
    'n_estimators': [75,100,125],
    'max_depth': [None, 3, 5, 7],
    'max_features': ['auto', 'log2']
}
gs = GridSearchCV(rf, params, n_jobs=8, verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed:  3.9min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
             param_grid={'max_depth': [None, 3, 5, 7],
                         'max_features': ['auto', 'log2'],
                         'n_estimators': [75, 100, 125]},
             verbose=1)

In [11]:
gs.best_params_

{'max_depth': None, 'max_features': 'auto', 'n_estimators': 125}

In [12]:
gs.score(X_train, y_train)

0.8317262053459905

In [13]:
gs.score(X_test, y_test)

0.539568345323741

Hmmm pretty significant overfit from this grid search. Training score was ~83% but testing was only 54%. I'm going to force the grid search into some hyperparameters that should decrease variance.

In [14]:
rf = RandomForestClassifier()
params = {
    'n_estimators': [150], 
    'max_depth': [100], #remove None from options for max depth
    'min_samples_split': [5],
    'max_features': ['auto']
}
gs = GridSearchCV(rf, params, n_jobs=8, verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   47.7s remaining:  1.2min
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   47.8s finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
             param_grid={'max_depth': [100], 'max_features': ['auto'],
                         'min_samples_split': [5], 'n_estimators': [150]},
             verbose=1)

In [15]:
gs.best_params_

{'max_depth': 100,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [16]:
gs.score(X_train, y_train)

0.6644016987259556

In [17]:
gs.score(X_test, y_test)

0.509742206235012

These grid-searches are taking 8-10 minutes each, I don't want to spend too much time tuning hyperparameters with other models to try as well. If I have time I may come back and try to improve this train/test disparity.

My next model will be K-nearest neighbors model.

In [18]:
knn = KNeighborsClassifier()
k_params = {
    'n_neighbors': [3,5,7,9],
    'p': [1,2]
}
gs_knn = GridSearchCV(knn, k_params, n_jobs=8, verbose=1)
gs_knn.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed:  2.7min finished


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=8,
             param_grid={'n_neighbors': [3, 5, 7, 9], 'p': [1, 2]}, verbose=1)

In [19]:
gs_knn.best_params_

{'n_neighbors': 3, 'p': 2}

In [20]:
gs_knn.score(X_train, y_train)

0.6613539845116163

In [21]:
gs_knn.score(X_test, y_test)

0.5040467625899281

My next model will be a Logistic Regression model. I need to reduct my 'y' to a single column so I'll have to map number values to each possible outcome.

In [22]:
y2 = train['outcometype']
y2 = y2.map({
    'Adoption':1,
    'Transfer':2,
    'Return_to_owner':3,
    'Euthanasia':4,
    'Died':5
})

In [23]:
y2.value_counts(normalize=True) #distribution matches prior value counts, looks good

1    0.403530
2    0.352231
3    0.179301
4    0.058156
5    0.006782
Name: outcometype, dtype: float64

In [24]:
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X,y2,random_state=17,stratify=y)

In [25]:
for num in [X_train_lr, X_test_lr, y_train_lr, y_test_lr]: #check shapes
    print(num.shape)

(20015, 1750)
(6672, 1750)
(20015,)
(6672,)


In [26]:
lr = LogisticRegression()
lr_params = {
    'penalty': ['l2'],
    'C': [.1,1],
    'solver':['sag', 'saga'],
    'max_iter':[150]
}
gs_lr = GridSearchCV(lr, lr_params, n_jobs=10, verbose=1)
gs_lr.fit(X_train_lr, y_train_lr)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  20 out of  20 | elapsed:  5.9min finished
C:\Users\willg\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


GridSearchCV(estimator=LogisticRegression(), n_jobs=10,
             param_grid={'C': [0.1, 1], 'max_iter': [150], 'penalty': ['l2'],
                         'solver': ['sag', 'saga']},
             verbose=1)

In [27]:
gs_lr.best_params_

{'C': 0.1, 'max_iter': 150, 'penalty': 'l2', 'solver': 'sag'}

In [28]:
gs_lr.score(X_train_lr, y_train_lr)

0.5137147139645266

In [29]:
gs_lr.score(X_test_lr, y_test_lr)

0.5187350119904077

My final model will be a linear support vector classifier.

In [30]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [31]:
lsvc = LinearSVC()
lsvc_params = {
    'dual':[True, False],
    'C':[.1,1],
    'max_iter': [1000],
}
gs_lsvc = GridSearchCV(lsvc, lsvc_params, n_jobs=10, verbose=1)
gs_lsvc.fit(X_train_lr, y_train_lr)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  20 out of  20 | elapsed:   18.7s finished


GridSearchCV(estimator=LinearSVC(), n_jobs=10,
             param_grid={'C': [0.1, 1], 'dual': [True, False],
                         'max_iter': [1000]},
             verbose=1)

In [32]:
gs_lsvc.best_params_

{'C': 0.1, 'dual': False, 'max_iter': 1000}

In [33]:
gs_lsvc.score(X_train_lr, y_train_lr)

0.6297776667499375

In [34]:
gs_lsvc.score(X_test_lr, y_test_lr)

0.6294964028776978

In [35]:
coefs = gs_lsvc.best_estimator_.coef_

In [36]:
coef_df = pd.DataFrame(coefs, columns=X_train_lr.columns)

In [37]:
coef_df.T.sort_values(by=0, ascending=False)

,0,1,2,3,4
sexuponoutcome_Spayed Female,1.315724,-0.975315,0.061369,-0.229145,-0.042535
sexuponoutcome_Neutered Male,1.210292,-0.951636,0.120183,-0.196930,-0.050287
color_Blue Tabby/White,0.097222,-0.152105,-0.038181,-0.018594,0.012577
color_Brown/Black,0.074100,-0.073833,-0.110893,0.115915,0.009667
breed_Queensland Heeler Mix,0.071933,-0.086139,-0.244696,0.090960,-0.014426
...,...,...,...,...,...
breed_Domestic Medium Hair Mix,-0.130076,0.023235,-0.107257,0.078545,-0.053482
breed_Shih Tzu Mix,-0.130253,0.283664,0.088140,-0.097686,-0.018219
breed_Pit Bull Mix,-0.250698,-0.066910,0.113659,0.231684,-0.010024
animaltype_Dog,-0.276264,-0.074473,0.287437,-0.071210,-0.089077


In [38]:
test = pd.read_csv('./data/test.csv.gz')
test.head() #preview test set

,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
2,3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
3,4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
4,5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White


In [39]:
test.columns = [col.lower().replace(' ', '_') for col in test.columns] #format columns

In [40]:
test.isnull().sum()

id                   0
name              3225
datetime             0
animaltype           0
sexuponoutcome       0
ageuponoutcome       6
breed                0
color                0
dtype: int64

In [41]:
train['name'].fillna('Unnamed', inplace=True)

Epilogue: this is where I would save and upload my kaggle test submission. Unfortunately I'm out of time.